In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os
from os.path import join as oj
import sys
sys.path.append('../src')
import numpy as np
import seaborn as sns
import torch
import matplotlib
from matplotlib import pyplot as plt
from sklearn import metrics
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import train_test_split
plt.style.use('dark_background')
import full_data_reg
import data
from skorch.callbacks import Checkpoint
from skorch import NeuralNetRegressor
from config import *
from tqdm import tqdm
import pickle as pkl
import train_reg
from math import floor
from copy import deepcopy
import config
import models
import pandas as pd
import features
from scipy.stats import skew, pearsonr
import outcomes
import neural_networks
from sklearn.model_selection import KFold
from torch import nn, optim
from torch.nn import functional as F
import acd
from acd.scores import cd_propagate
import lime, shap

# prepare data

In [1]:
# currently trained only on 'clath_aux+gak_a7d2_new'
dsets = ['clath_aux+gak_new']
splits = ['test']
#feat_names = ['X_same_length_normalized'] + data.select_final_feats(data.get_feature_names(df))
              #['mean_total_displacement', 'mean_square_displacement', 'lifetime']
meta = ['cell_num', 'Y_sig_mean', 'Y_sig_mean_normalized']
dfs = {}
for dset in tqdm(dsets):
    for split in splits:
        df = data.get_data(dset=dset)
        df = df[~(df.short | df.long | df.hotspots)]
#         df = df[df.valid]
        df = df[df.lifetime > 15] # only keep hard tracks
        df = df[df.cell_num.isin(config.DSETS[dset][split])] # exclude held-out test data
        feat_names = ['X_same_length_normalized'] + data.select_final_feats(data.get_feature_names(df))

        # downsample tracks
        length = 40
        df['X_same_length'] = [features.downsample(df.iloc[i]['X'], length)
                               for i in range(len(df))] # downsampling
        # normalize tracks
        df = features.normalize_track(df, track='X_same_length', by_time_point=False)

        # regression response
        df = train_reg.add_sig_mean(df)     

        # remove extraneous feats
        # df = df[feat_names + meta]
#         df = df.dropna() 

        # normalize features
        for feat in feat_names:
            if 'X_same_length' not in feat:
                df = features.normalize_feature(df, feat)

        dfs[(dset, split)] = deepcopy(df)

# load model
# results = pkl.load(open('../models/dnn_full_long_normalized_across_track_4_feats.pkl', 'rb'))
# dnn = neural_networks.neural_net_sklearn(D_in=40, H=40, p=3, arch='lstm')
results = pkl.load(open('../models/clath_aux+gak_a7d2_new_Y_sig_mean_normalized_nn_lstm_100_40.pkl', 'rb'))
dnn = neural_networks.neural_net_sklearn(D_in=40, H=40, p=17, arch='lstm')
dnn.model.load_state_dict(results['model_state_dict'])

NameError: name 'tqdm' is not defined

# prepare for interp

In [8]:
# df = dfs[('clath_aux+gak_a7d2', 'train')]
df = dfs[('clath_aux+gak_new', 'test')]
X = df[feat_names]
y = df['y_consec_thresh']


# interpret shifts

In [ ]:
fdsklj

# important features

In [ ]:
lime_explanation = explainer.explain_instance(x, m.predict_proba, num_features=x.size) #, top_labels=1)
# lime_explanation.show_in_notebook(show_table=True)
# lime_explanation.as_list() # give more info on fitted lime values
lime_values = [l[1] for l in lime_explanation.as_list()]

# global feat imp

In [13]:
def perm_importance(X, model, feat_names):
    '''custom bc some features are lists (e.g. the track)'''
    acc_orig = np.mean((dnn.predict(X) > 0) == y)
    imps = {feat_name: [] for feat_name in feat_names}
    reps = 3
    for feat_name in tqdm(feat_names):
        for r in range(reps):
            X_copy = deepcopy(X)
            X_copy[feat_name] = np.random.permutation(X_copy[feat_name].values)
            acc = np.mean((dnn.predict(X_copy) > 0) == y)
            imps[feat_name].append(acc_orig - acc)
    return pd.DataFrame.from_dict(imps).mean().sort_values(ascending=False)

np.random.seed(13)
imps = perm_importance(X, dnn, feat_names)